In [35]:
import mysql.connector
import pandas as pd
import re

In [3]:
# Replace these with your actual credentials
db_config = {
    'user': 'root',
    'password': '',
    'host': 'localhost',
    'database': 'job_scraping',
}

# Establish a connection to the MySQL database
connection = mysql.connector.connect(**db_config)


In [49]:
query = 'SELECT JobId, Salary FROM marketing_jobs_linkedin where Salary <> "";'

cursor = connection.cursor()
cursor.execute(query)
result = cursor.fetchall()

df = pd.DataFrame(result, columns = cursor.column_names)

In [50]:
salary = df["Salary"].str.replace(",", "").str.split(" - ")

In [51]:
# Loop through each sublist in the data
for sublist_index in range(len(salary)):
    for element_index in range(len(salary[sublist_index])):
        # Replace "/yr" with ""
        salary[sublist_index][element_index] = salary[sublist_index][element_index].replace("/yr", "")
        # Replace "/hr" with ""
        salary[sublist_index][element_index] = salary[sublist_index][element_index].replace("/hr", "")
        # Replace "/month" with ""
        salary[sublist_index][element_index] = salary[sublist_index][element_index].replace("/month", "")
        # Replace "$" with ""
        salary[sublist_index][element_index] = salary[sublist_index][element_index].replace("$", "")


In [18]:
salary_df = pd.DataFrame()
salary_df["id"] = df["JobId"]

In [19]:
salary_type = []
for i in df["Salary"]:
    if "hr" in i:
        salary_type.append("hr")
    elif "month" in i:
        salary_type.append("month")
    else:
        salary_type.append("yr")
        
salary_df["salary_type"] = salary_type

In [47]:
min_salary = []
max_salary = []
for job in salary:
    if len(job) == 1:
        min_salary.append(float(job[0]))
        max_salary.append(float(job[0]))
    else:
        min_salary.append(float(job[0]))
        max_salary.append(float(job[1]))

salary_df["min"] = min_salary
salary_df["max"] = max_salary

In [65]:
# Remove duplicates based on the "id" column
salary_df.drop_duplicates(subset=["id"], keep="first", inplace=True)


In [66]:
# cursor.execute('''
#                 CREATE TABLE salary (
#                     id varchar(100) primary key not null,
#                     salary_type varchar(10),
#                     min_salary decimal(10, 2),
#                     max_salary decimal(10, 2)
#                 )
# ''')

In [70]:
insert_query = "INSERT IGNORE INTO salary (id, salary_type, min_salary, max_salary) VALUES (%s, %s, %s, %s)"

# Iterate over each row in the DataFrame and insert the values into the table
for _, row in salary_df.iterrows():
    values = (
        row["id"],
        row["salary_type"],
        row["min"],
        row["max"]
    )
    cursor.execute(insert_query, values)

# Commit the changes to the database
# connection.commit()

# # Close the cursor and database connection
# cursor.close()
# mydb.close()